In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [2]:
train = pd.read_csv("D:/kaggle_data/quora-question-pairs/train.csv")
test = pd.read_csv("D:/kaggle_data/quora-question-pairs/test.csv")
sample_submission = pd.read_csv("D:/kaggle_data/quora-question-pairs/sample_submission.csv")

In [3]:
train.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [4]:
train = train.drop(['id', 'qid1', 'qid2'], axis=1)

In [5]:
train.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [7]:
test = test.drop(['test_id'], axis=1)

In [8]:
train = train.fillna('null')

In [9]:
test = test.fillna('null')

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer = 'word', stop_words = 'english', lowercase = True, norm = 'l2')

In [11]:
train_q1_tfidf = tfidf.fit_transform(train.question1.values)
train_q2_tfidf = tfidf.fit_transform(train.question2.values)

In [12]:
test_q1_tfidf = tfidf.fit_transform(test.question1.values)
test_q2_tfidf = tfidf.fit_transform(test.question2.values)

In [13]:
train_q1_tfidf.shape

(404290, 67533)

In [14]:
train_q2_tfidf.shape

(404290, 62375)

In [15]:
#행렬 형태 통일
train_q1_tfidf = csr_matrix((train_q1_tfidf.data, train_q1_tfidf.indices, train_q1_tfidf.indptr), shape=(404290,90824))
train_q2_tfidf = csr_matrix((train_q2_tfidf.data, train_q2_tfidf.indices, train_q2_tfidf.indptr), shape=(404290,90824))

In [16]:
test_q1_tfidf.shape


(2345796, 90824)

In [17]:
test_q2_tfidf.shape

(2345796, 90765)

In [18]:
#행렬 형태 통일
test_q2_tfidf = csr_matrix((test_q2_tfidf.data, test_q2_tfidf.indices, test_q2_tfidf.indptr), shape=(2345796,90824))

In [19]:
X_train = abs(train_q1_tfidf-train_q2_tfidf)
y = train[['is_duplicate']]

In [20]:
X_test = abs(test_q1_tfidf-test_q2_tfidf)

In [21]:
#XGB classifier 사용
from xgboost import XGBClassifier

xg_model = XGBClassifier()
xg_model.fit(X_train, y)
xg_pred = xg_model.predict(X_test)


C:\Users\Min\AppData\Roaming\Python\Python37\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Min\AppData\Roaming\Python\Python37\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
xg_pred = pd.Series(xg_pred, name='is_duplicate')
submission = pd.concat([pd.Series(range(2345796), name='test_id'),xg_pred], axis = 1)
submission.to_csv('submission_file.csv', index=False)
